 # Setup Workspace and Experiment

In [ ]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

run = exp.start_logging()

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "compute-cluster-analytics"

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

try:
    compute = ComputeTarget(ws, cluster_name)
    print("Found existing cluster")
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size="Standard_D2_V2", max_nodes=4)
    compute = ComputeTarget.create(ws, cluster_name, compute_config)
    print("Created a new cluster")

compute.wait_for_completion()
print("Cluster ready")

 # Hyperparameter Tuning

In [ ]:
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler
ps = RandomParameterSampling({"--C": uniform(0.01, 0.99)})

# Specify a Policy
policy = BanditPolicy(evaluation_interval=50, slack_factor=0.3)

if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig(".", 
    script="train.py", 
    compute_target=compute, 
    environment=sklearn_env, 
    arguments=["--max_iter", 200])

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(run_config=src,
    hyperparameter_sampling=ps,
    policy=policy, 
    primary_metric_name="Accuracy", 
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE, 
    max_total_runs=20)

In [ ]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

run = exp.submit(hyperdrive_config)

In [ ]:
run = ws.get_run('HD_1d3a32ee-7a81-45d6-98ac-4c63a34cfbed')

In [ ]:
from azureml.widgets import RunDetails
RunDetails(run).show()

In [ ]:
import joblib
# Get your best run and save the model from that run.

best_run = run.get_best_run_by_primary_metric()
best_run.get_details()

In [ ]:
# best_run.register_model(model_name="best_random_sample_c_logistic_regression", model_path="outputs/model.pkl")
#
# to save the best model it would need to be an output of the training script

 # Auto ML

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

path = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
ds = TabularDatasetFactory.from_delimited_files(path)

In [ ]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)
data = x.join(y)
data

In [ ]:
import os 
local_dir = "data"
os.makedirs(local_dir, exist_ok=True)
file_name = "cleaned_data.jsonl"
data.to_json(f"./{local_dir}/{file_name}", orient="records", lines=True)

In [ ]:
from azureml.core import Dataset
# register dataset
datastore = ws.get_default_datastore()
datastore_path = "/users/mdelwig/udacity/project1/data"
datastore.upload("data", datastore_path, overwrite=True)
dataset = Dataset.Tabular.from_json_lines_files((datastore, f"{datastore_path}/{file_name}"))
dataset_name = "udacity_project1_data"
dataset.register(ws, dataset_name)

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=dataset,
    label_column_name='y',
    n_cross_validations=5,
    compute_target=compute)

In [ ]:
# Submit your automl run

ml_run = exp.submit(automl_config)

In [ ]:
ml_run = ws.get_run("AutoML_e647946e-3a69-43d3-be2b-cad2e5f3c5fe")

In [ ]:
RunDetails(ml_run).show()

In [ ]:
# Retrieve and save your best automl model.

best_ml_run = ml_run.get_best_child()
best_ml_run.get_details()

In [ ]:
best_ml_run.register_model(model_name="udacity1_best_ml_model", model_path="outputs/model.pkl")

In [ ]:
try:
    compute = ComputeTarget(ws, cluster_name)
    compute.delete()
    print("Deleted cluster")
except ComputeTargetException:
    print("Cluster allready deleted")